In [1]:
from bs4 import BeautifulSoup
from glob import glob
import json
import os
import re
import nltk

In [2]:
# prep work
fnames = []
for root, dirs, files in os.walk("pdf"):
    for fn in files:
        if fn.endswith('.json'):
            base = os.path.join(root, ''.join(fn.split('.')[:-1]))
            if os.path.isfile(base+'.json') and os.path.isfile(base+'.tei.xml'):
                fnames.append(os.path.join(root, fn.replace('tei.xml', 'json')))
            
DBDOI = {}
for fn in fnames:
    with open(fn, 'r') as fp:
        data = json.load(fp)
        if data['abstract'] is not None:
            DBDOI[data['bibcode']] = {
                'fname': fn,
                'title': data['title']
            }
print(len(DBDOI))

1711


In [3]:
def get_references(fname, title, ctx=100):
    
    with open(fn_rawtext, 'r') as fp:
        raw = fp.read()
    rawsoup = BeautifulSoup(raw, 'lxml')
    
    bibid = []
    for bib in rawsoup.body.find_all('biblstruct'):
        if 'xml:id' in bib.attrs.keys():
            #print(title, bib.title.get_text())
            if title == bib.title.get_text():
                bibid = bib.attrs['xml:id']
    if len(bibid) == 0:
        return []
        
    pos = re.finditer(bibid, str(rawsoup.body))
    print(len(list(pos)))
    pos = re.finditer(bibid, str(rawsoup.body))

    refs = []
    for p in pos:
        rawctx = str(rawsoup.body)[max(0,p.start()-ctx):min(len(str(rawsoup.body)),p.end()+ctx)]
        soup = BeautifulSoup(rawctx, 'lxml')

        if 'biblstruct' in rawctx or len(soup.get_text().strip()) == 0:
            # this is the bibliography entry
            continue
            
        datablock = {
            'paperid': summary['bibcode'],
            'context': soup.get_text(),
            'cited': soup.find_all("ref", attrs={"type":"bibr", "target":"#%s" % bibid})[0].get_text(),
            'otherid': [idx for idx in summary['reference'] if idx in DBDOI.keys()]
        }
        refs.append(datablock)
    return refs

In [4]:
# Case 1: provide a doi
# Case 2: provide a search term

In [5]:
# Case 1: provide a doi
target = "10.1002/2013JE004555"
target = "10.1002/jgre.20103"
#target = "10.1089/ast.2017.1700"

with open("pdf/"+target+".json", "r") as f:
    summary = json.load(f)

print("Title: %s" % summary['title'])
print("Abstract: %s" % summary['abstract'])
print("References: %d" % len(summary['reference']))
print("Citations: %d" % len(summary['citation']))

Title: Asymmetric thermal evolution of the Moon
Abstract: The Moon possesses a clear dichotomy in geological processes between the nearside and farside hemispheres. The most pronounced expressions of this dichotomy are the strong concentration of radioactive heat sources on the nearside in a region known as the Procellarum KREEP Terrane (PKT) and the mare basaltic lava flows that erupted in or adjacent to this terrane. We model the thermochemical evolution of the Moon using a 3-D spherical thermochemical convection code in order to assess the consequences of a layer enriched in heat sources below the PKT on the Moon's global evolution. We find that in addition to localizing most of the melt production on the nearside, such an enriched concentration of heat sources in the PKT crust has an influence down to the core-mantle boundary and leaves a present-day temperature anomaly within the nearside mantle. Moderate gravitational and topographic anomalies that are predicted in the PKT, but n

In [6]:
# citing papers
for citing in summary['citation']:
    if citing in DBDOI.keys():
        print("Citing paper: %s" % DBDOI[citing]['title'])
        fn_rawtext = DBDOI[citing]['fname'][:-5] + '.tei.xml'
        res = get_references(DBDOI[citing]['fname'], summary['title'], ctx=200)

        if len(res) > 0:
            for c in res:
                
                idx = c['context'].find(c['cited'])
                if re.match('^[^a-zA-Z]', c['cited']):
                    idx += 1
                print("{:>70s}{}{:<70s}".format(c['context'][max(0, idx-70):idx], 
                                                c['context'][idx:idx+len(c['cited'])], 
                                                c['context'][idx+len(c['cited']):min(len(c['context']),idx+len(c['cited'])+70)]))
            print()

Citing paper: Structure and evolution of the lunar Procellarum region as revealed by GRAIL gravity data
12
m the high crustal concentrations of heat-producing elements [10][11][12] . The region encompasses the majority of the Moon's mare basalt provin
 lithosphere. Although early convection beneath the PKT was possible 12 , this convection would have had only a second-order effect on the tem
ured a 10-km-thick layer of KREEP basalt at the base of the crust 10,12 , but other workers have argued that this scenario is not compatible w
 0.14 p.p.m. and 6.8 p.p.b. in the crust and mantle, respectively 10,12 . We assumed a K/U ratio of 2,500 and a Th/U ratio of 3.7 in all mater
                                   crust and mantle, respectively 10,12 . We assumed a K/U ratio of 2,500 and a Th/U ratio of 3.7 in all mater
fferent ratios between the concentrations of heat-producing elements 12 and the neglect of latent heat and melt extraction effects in this stu
sistence of mantle convection thr

IndexError: list index out of range